# Map

In [28]:
import random

In [29]:
OCEAN=0
MOUNTAIN='M'
PRISON='P'
TREASURE='T'

In [84]:
class MapGenerator:
    def __init__(self,size,num_of_region,num_of_prison,num_of_mountain):
        self.map=[]
        self.size=size
        self.num_of_region=num_of_region
        self.num_of_prison=num_of_prison
        self.num_of_mountain=num_of_mountain
        
        for i in range(self.size):
            self.map.append([])
            for j in range(self.size):
                self.map[i].append(-1)
                
        self.create_ocean()
        self.create_region()
        self.allocate_mountain()
        self.pirate_pos=self.allocate_prison()
        self.treasure_pos=self.allocate_treasure()
    def create_ocean(self):
        prob1=0.5
        prob2=0.1
        for i in range(len(self.map)):
            for j in range(len(self.map[i])):
                if i==0 or i==len(self.map)-1 or j==0 or j==len(self.map)-1:
                    self.map[i][j]=OCEAN
                    expand=random.randint(0,1)
                    self.expand_prob(i,j,prob1)
                    if self.size//10>=3:
                        for x in [-1,0,1]:
                            for y in [-1,0,1]:
                                if x==0 and y==0: continue
                                try:
                                    if self.map[i+x][j+y]==OCEAN:
                                        self.expand_prob(i+x,j+y,prob2)
                                except: pass
                            
    def expand_prob(self,i,j,prob):
#         expand with expanded probability 0->1
        value=self.map[i][j] 
        prob=[1]*int(prob*10)+[0]*int((1-prob)*10)
        expand=random.choice(prob)
        if expand:
            for x in [-1,0,1]:
                for y in [-1,0,1]:
                    if x==0 and y==0: continue
                    if x!=0 and y!=0: continue
                    try:
                        self.map[i+x][j+y]=max(self.map[i+x][j+y],value)
                    except: pass
    def divide_region(self):
        region_ratio=[]
        for i in range(self.num_of_region-1):
            rate=random.choice([1/self.num_of_region,1/self.num_of_region+1/self.num_of_region/5])
            region_ratio.append(rate)
        region_ratio.append(1-sum(region_ratio))         
        total=self.size**2
        ocean_count=0
        for i in range(len(self.map)):
            for j in range(len(self.map[i])):
                if self.map[i][j]==OCEAN:
                    ocean_count+=1
        total-=ocean_count
        region_tile={}
        for i in range(self.num_of_region):
            region_tile[i+1]=round(total*region_ratio[i])
        if sum(region_tile.values())+ocean_count!=self.size**2:
            region_tile[self.num_of_region]+=self.size**2-sum(region_tile.values())-ocean_count
        
        return region_tile
                    
    def visualize (self):
        for i in range(len(self.map)):
            for j in range(len(self.map[i])):
                print('{:>2}'.format(self.map[i][j]),end=' ')
            print()
    def expand_tile(self,i,j,region,tile): #expand with number of tile
        queue=[[i,j]]
        while tile>0 and len(queue)>0:
            index=queue.pop(0)
            self.map[index[0]][index[1]]=region
            tile-=1
            i,j=index[0],index[1]
            for x in [-1,0,1]:
                for y in [-1,0,1]:
                    if x==0 and y==0: continue
                    if self.map[i+x][j+y]==-1 and [i+x,j+y] not in queue:
                        queue.append([i+x,j+y])

        

    def create_region(self):
        region_tile=self.divide_region()
        for region in region_tile.keys():
            region_flag=False
            for i in range(len(self.map)):
                for j in range(len(self.map[i])):
                    if self.map[i][j]==-1:
                        self.expand_tile(i,j,region,region_tile[region])
                        region_flag=True
                        break
                if region_flag:
                    break
        # final check
        for i in range(len(self.map)):
                for j in range(len(self.map[i])):
                    if self.map[i][j]==-1:
                        self.map[i][j]=self.map[i-1][j]
    def allocate_mountain(self):
        moutain_length=min(self.size//self.num_of_mountain,10)
        for i in range(self.num_of_mountain):
            count=moutain_length
            x=0
            y=0
            while self.map[x][y]==0 or isinstance(self.map[x][y], str) :
                x=random.randint(0,self.size-1)
                y=random.randint(0,self.size-1)
            self.map[x][y]=str(self.map[x][y])+MOUNTAIN
            count-=1
            a=b=0
            while count>0:
                visited=[]
                while (a!=0 and b!=0) or self.map[x+a][y+b]==0 or isinstance(self.map[x+a][y+b], str):
                    tmp_a=random.choice([-1,1,0])
                    tmp_b=random.choice([-1,1,0])
                    if [tmp_a,tmp_b] in visited:
                        continue
                    else:
                        visited.append([a,b])
                        a,b=tmp_a,tmp_b
                x=x+a
                y=y+b
                self.map[x][y]=str(self.map[x][y])+MOUNTAIN
                count-=1
                a=b=1
                   
    def allocate_treasure(self):
        x=0
        y=0
        while self.map[x][y]==0 or isinstance(self.map[x][y], str) :
            x=random.randint(0,self.size-1)
            y=random.randint(0,self.size-1)
        self.map[x][y]=str(self.map[x][y])+TREASURE
        return [x,y]
    def allocate_prison(self):
        for i in range(self.num_of_prison):
            x=0
            y=0
            while self.map[x][y]==0 or isinstance(self.map[x][y], str) :
                x=random.randint(0,self.size-1)
                y=random.randint(0,self.size-1)
            self.map[x][y]=str(self.map[x][y])+PRISON
        return [x,y] #the position of the pirate
        
    def export(self,filename):
        with open (filename,'a') as f:
            f.write(f'{self.size} {self.size}\n') #size
            if self.size//10>3:
                f.write(f'{random.randint(2,4)}\n') #turn reveal prison
            else: 
                f.write(f'{random.randint(3,4)}\n')
            f.write(f'{random.randint(self.size/2,self.size/2+self.size//10)}\n') #turn release pirate
            f.write(f'{self.num_of_region}\n')
            f.write(f'{self.treasure_pos[0]} {self.treasure_pos[1]}\n')
#             map
            for i in range(len(self.map)):
                for j in range(len(self.map[i])):
                    if j==len(self.map[i])-1:
                        f.write('{:>2}\n'.format(self.map[i][j]))
                    else:
                        f.write('{:>2};'.format(self.map[i][j]))

            
            

In [91]:
m=MapGenerator(size=16,num_of_region=5,num_of_prison=3,num_of_mountain=3)
m.visualize()

 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 
 0  0  0  0  1  1  1  0  0  1  0  0  0  2  0  0 
 0  0  1  1  1  1  1  1  1  1  2  2 2M 2M  2  0 
 0  0  1 1P  1 1P  1  1  1  1  2  2  2 2M  2  0 
 0  1  1  1 1T  1  1 1M 1M  1  2  2 2M 2M  2  0 
 0  0  1  1  1  1  1 1M 1M  1  2  2  2  2  2  0 
 0  0  1  1  1  3  3  2 2M  2  2  2  2  2  0  0 
 0  0  3  3  3  3  3  2  2  2  2  2  2  2  2  0 
 0  3  3  3  3  3  3  2 2M 2M  2  4  4  4  0  0 
 0  0  3  3  3  3  3  3  3 4M  4  4  4  4  4  0 
 0  0  3  3  3  3  3  3  3 4M 4M  4  4  4  0  0 
 0  0  3  3  3  3  3  3  4  4  4  4  4  4  4  0 
 0  0  5  5  5  5  4  4  4  4  4  4  4  4  0  0 
 0  0  5  5  5  5  4  4  4  4  5  5  5  5  5  0 
 0  0  0  0  0  0  5  5 5P  5  0  5  0  5  0  0 
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 


In [92]:
m.export('MAP02.txt')

In [107]:
t.visualize_map()

 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 
 0  0  0  0  1  1  1  0  0  1  0  0  0  2  0  0 
 0  0  1  1  1  1  1  1  1  1  2  2 2M 2M  2  0 
 0  0  1 1P  1 1P  1  1  1  1  2  2  2 2M  2  0 
 0  1  1  1 1T  1  1 1M 1M  1  2  2 2M 2M  2  0 
 0  0  1  1  1  1  1 1M 1M  1  2  2  2  2  2  0 
 0  0  1  1  1  3  3  2 2M  2  2  2  2  2  0  0 
 0  0  3  3  3  3  3  2  2  2  2  2  2  2  2  0 
 0  3  3  3  3  3  3  2 2M 2M  2  4  4  4  0  0 
 0  0  3  3  3  3  3  3  3 4M  4  4  4  4  4  0 
 0  0  3  3  3  3  3  3  3 4M 4M  4  4  4  0  0 
 0  0  3  3  3  3  3  3  4  4  4  4  4  4  4  0 
 0  0  5  5  5  5  4  4  4  4  4  4  4  4  0  0 
 0  0  5  5  5  5  4  4  4  4  5  5  5  5  5  0 
 0  0  0  0  0  0  5  5 5P  5  0  5  0  5  0  0 
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 


# Hint

In [5]:
import numpy as np
import random

# Hint 1: [0, tuple (x, y - x: row, y: col)]: A list of random tiles that doesn't contain the treasure (1 to 12).
# Hint 2: [1, 2 - 5 số từ 1 - 5]: 2-5 regions that 1 of them has the treasure.
# Hint 3: [2, 2 - 3 số từ 1 - 5]: 1-3 regions that do not contain the treasure.
# Hint 4: [3, top, bottom, left, right]: A large rectangle area that has the treasure. (< half: small; >= half: large)
# Hint 5: [4, top, bottom, left, right]: A small rectangle area that doesn't has the treasure. (< half: small; >= half: large)
# Hint 6: [5]: He tells you that you are the nearest person to the treasure (between you and the prison he is staying).
# Hint 7: [6, 0: Row/ 1: Column, x]: A column and/or a row that contain the treasure (rare)
# Hint 8: [7, 0: Row/ 1: Column, x]: A column and/or a row that do not contain the treasure.
# Hint 9: [8, x, y]: 2 regions that the treasure is somewhere in their boundary
# Hint 10: [9]: The treasure is somewhere in a boundary of 2 regions.
# Hint 11: [10, 2/3]: The treasure is somewhere in an area bounded by 2-3 tiles from sea.
# Hint 12: [11, 1: Half right / 2: Half left / 3: Half top / 4: Half bottom]: A half of the map without treasure (rare).
# Hint 13: [12, 1: Center / 2: Prison, 1: N / 2: S / 3: W / 4: E / 5: SE / 6: SW / 7: NE / 8: NW]: From the center of the map/from the prison that he's staying, 
  # he tells you a direction that has the treasure 
  # (W, E, N, S or SE, SW, NE, NW) 
  # (The shape of area when the hints are either W, E, N or S is triangle).
# Hint 14: [13, top_big, bottom_big, left_big, right_big, top_small, bottom_small, left_small, right_small]: 
  # 2 squares that are different in size, the small one is placed inside the bigger one, the treasure is somewhere inside the gap between 2 squares. (rare)
# Hint 15: [14]: The treasure is in a region that has mountain.

def generateHint(map, treasure_row, treasure_col, regions, treasure_region, agent_row, agent_col, prison_row, prison_col):
  hint = []
  logMsg = ""
  # p = 0.045 for hints that are rare
  index = np.random.choice(a=np.arange(0, 15, 1, dtype=int), p = (0.07, 0.07, 0.07, 0.07, 0.07, 0.07, (1-0.07*12)/3, 0.07, 0.07, 0.07, 0.07, (1-0.07*12)/3, 0.07, (1-0.07*12)/3, 0.07))
  # index = 3

  if index == 0:

    # Hint 1: [0, tuple (x, y - x: row, y: col)]: A list of random tiles that doesn't contain the treasure (1 to 12).

    hint = generateHint1()

    logMsg = "These tile(s) do not contain the treasure: " + ', '.join(map(str, hint[1:]))

  elif index == 1:

    # Hint 2: [1, 2 - 5 số từ 1 - 5]: 2-5 regions that 1 of them has the treasure.

    hint = generateHint2(regions)

    logMsg = "Treasure is in one of these region(s): " + ', '.join(map(str, hint[1:]))

  elif index == 2:

    # Hint 3: [2, 2 - 3 số từ 1 - 5]: 1-3 regions that do not contain the treasure.

    hint = generateHint3(regions)

    logMsg = "These region(s) do not contain the treasure: " + ', '.join(map(str, hint[1:]))

  elif index == 3:

    # Hint 4: [3, top, bottom, left, right]: A large rectangle area that has the treasure. (< half: small; >= half: large)

    hint = generateHint4(map)

    logMsg = "This area contains the treasure: From row "+str(hint[1])+" to row "+str(hint[2])+", from column "+str(hint[3])+" to column "+str(hint[4])
      
  elif index == 4:

    # Hint 5: [4, top, bottom, left, right]: A small rectangle area that doesn't has the treasure. (< half: small; >= half: large)

    hint = generateHint5(map)

    logMsg = "This area does not contain the treasure: From row "+str(hint[1])+" to row "+str(hint[2])+", from column "+str(hint[3])+" to column "+str(hint[4])

  elif index == 5:

    # Hint 6: [5]: He tells you that you are the nearest person to the treasure (between you and the prison he is staying).

    hint = generateHint6()

    logMsg = "You are the nearest person to the treasure (between you and the prison the pirate is staying)"

  elif index == 6:

    # Hint 7: [6, 0: Row/ 1: Column, x]: A column and/or a row that contain the treasure (rare)

    hint = generateHint7(map)

    if (hint[1] == 0): logMsg = "Row "
    else: logMsg = "Column "

    logMsg += str(hint[2]) + " contains the treasure"

  elif index == 7:

    # Hint 8: [7, 0: Row/ 1: Column, x]: A column and/or a row that do not contain the treasure.

    hint = generateHint8(len(map))

    if (hint[1] == 0): logMsg = "Row "
    else: logMsg = "Column "

    logMsg += str(hint[2]) + " does not contain the treasure"

  elif index == 8:

    # Hint 9: [8, x, y]: 2 regions that the treasure is somewhere in their boundary

    hint = generateHint9(map)

    logMsg += "Treasure is somewhere in these two regions' boudaries: Region " + str(hint[1]) + " and region " + str([hint[2]])

  elif index == 9:

    # Hint 10: [9]: The treasure is somewhere in a boundary of 2 regions.

    hint = generateHint10()

    logMsg += "The treasure is somewhere in a boundary of 2 regions"

  elif index == 10:

    # Hint 11: [10, 2/3]: The treasure is somewhere in an area bounded by 2-3 tiles from sea.

    hint = generateHint11()

    logMsg += "The treasure is somewhere in an area bounded by 2-3 tiles from sea"

  elif index == 11:
    
    # Hint 12: [11, 1: Half right / 2: Half left / 3: Half top / 4: Half bottom]: A half of the map without treasure (rare)

    hint = generateHint12()

    if hint[1] == 1:
      logMsg = "A half right"
    elif hint[1] == 2:
      logMsg = "A half left"
    elif hint[1] == 3:
      logMsg = "A half top"
    else:
      logMsg = "A half bottom"

    logMsg += " of the map does not have treasure"

  elif index == 12:

    # Hint 13: [12, 1: Center / 2: Prison, 1: N / 2: S / 3: W / 4: E / 5: SE / 6: SW / 7: NE / 8: NW]: From the center of the map/from the prison that he's staying, 
      # he tells you a direction that has the treasure 
      # (W, E, N, S or SE, SW, NE, NW) 
      # (The shape of area when the hints are either W, E, N or S is triangle).

    hint = generateHint13()

    direction, starting_point

    if hint[1] == 1: starting_point = "center of the map"
    else: starting_point = "Prison"

    if hint[2] == 1: direction = "North"
    elif hint[2] == 2: direction = "South"
    elif hint[2] == 3: direction = "West"
    elif hint[2] == 4: direction = "East"
    elif hint[2] == 5: direction = "South East"
    elif hint[2] == 6: direction = "South West"
    elif hint[2] == 7: direction = "North East"
    else: direction = "North West"

    logMsg += "From the "+starting_point+" that pirate's staying, the treasure is in the " + direction+ " direction"

  elif index == 13:

    # Hint 14: [13, top_big, bottom_big, left_big, right_big, top_small, bottom_small, left_small, right_small]: 
      # 2 squares that are different in size, the small one is placed inside the bigger one, the treasure is somewhere inside the gap between 2 squares. (rare)

    hint = generateHint14()

    logMsg = "the treasure is somewhere inside the gap between 2 squares:\n\t"
    logMsg += "The bigger square bounded by row "+hint[1]+" on top; row " +hint[2]+" at the bottom; "
    logMsg += "column "+hint[3]+" on the left and column "+hint[4]+ " on the right\n\t"
    logMsg += "The smaller square bounded by row "+hint[5]+" on top; row " +hint[6]+" at the bottom; "
    logMsg += "column "+hint[7]+" on the left and column "+hint[8]+ " on the right\n\t"

  elif index == 14:
    # Hint 15: [14]: The treasure is in a region that has mountain.
    logMsg = "The treasure is in a region that has mountain."

  else:
    pass

  return hint, logMsg

In [6]:
def generateHint1():
  hint = [0]
  num = random.randint(1,12)

  for i in range(num):
    row_index = random.randint(0,len(map) - 1)
    col_index = random.randint(0,len(map) - 1)
    hint.append((row_index, col_index))

  return hint

def verify_hint_1 (hint: list, treasurePos):
  # Hint 1: [0, tuple (x, y - x: row, y: col)]: A list of random tiles that doesn't contain the treasure (1 to 12).
  x, y = treasurePos
  for i in range(1, len(hint)):
    if (x, y) == hint[i]:
      return False
    return True

In [7]:
def generateHint2(regions):
  num = random.randint(2, 5)
  hint = [1]

  for i in range(num):
    random_region = random.randint(0, len(regions))
    if (random_region in hint):
      i-=1
      continue
    hint.append(random_region)

  return hint

def verify_hint_2 (hint: list, treasure_region):
  # Hint 2: [1, 2 - 5 số từ 1 - 5]: 2-5 regions that 1 of them has the treasure.
  for i in range(1, len(hint)):
    if hint[i] == treasure_region:
      return True
  return False

In [8]:
def generateHint3(regions):
  num = random.randint(2, 3)
  hint = [2]

  for i in range(num):
    random_region = random.randint(0, len(regions))
    if (random_region in hint):
      i-=1
      continue
    hint.append(random_region)

  return hint

def verify_hint_3 (hint: list, treasure_region):
  # Hint 3: [2, 2 - 3 số từ 1 - 5]: 1-3 regions that do not contain the treasure.
  for i in range(1, len(hint)):
    if hint[i] == treasure_region:
      return False
  return True

In [9]:
def generateHint4(map):
  hint = [3]
  area = random.randint(int(len(map)/2), len(map) - 1)
  random_row = random.randint(0,len(map) - 1)
  random_col = random.randint(0,len(map) - 1)
  # print(area)
  left = random_col - area
  if (left < 0): left = 0
  right = random_col + area
  if (right >= len(map)): right = len(map) - 1
  top = random_row - area
  if (top < 0): top = 0
  bottom = random_row + area
  if (bottom >= len(map)): bottom = len(map) - 1

  return [3, top, bottom, left, right]

def verify_hint_4 (hint: list, treasurePos):
  x, y = treasurePos
  # Hint 4: [3, top, bottom, left, right]: A large rectangle area that has the treasure. (< half: small; >= half: large)
  if x - hint[1] < 0:
    return False
  if hint[2] - x < 0:
    return False
  if y - hint[3] < 0:
    return False
  if hint[4] - y < 0:
    return False
  return True

In [10]:
def generateHint5(map):
  no_treasure_row = random.randint(0,len(map) - 1)
  no_treasure_col = random.randint(0,len(map) - 1)

  # print([no_treasure_row, no_treasure_col])

  area = random.randint(0,int(len(map) / 4) -1)
  # print(np.arange(0, area_expand, 1, dtype=int))
  # print(area)
  left = no_treasure_col - area
  if (left < 0): left = 0
  right = no_treasure_col + area
  if (right >= len(map)): right = len(map) - 1
  top = no_treasure_row - area
  if (top < 0): top = 0
  bottom = no_treasure_row + area
  if (bottom >= len(map)): bottom = len(map) - 1

  return [4, top, bottom, left, right]

def verify_hint_5 (hint: list, treasurePos):
  # Hint 5: [4, top, bottom, left, right]: A small rectangle area that doesn't has the treasure. (< half: small; >= half: large)
  x, y = treasurePos
  if x >= hint[1] and hint[2] >= x and y >= hint[3] and hint[4] >= y:
    return False
  return True

In [11]:
def generateHint6():
  return [5]

def verify_hint_6 (hint: list, treasurePos, agentPos, prisonPos):
  # Hint 6: [5]: He tells you that you are the nearest person to the treasure (between you and the prison he is staying).
  x, y = treasurePos
  agent_x, agent_y = agentPos
  prison_x, prison_y = prisonPos
  agent_dist = abs(y - agent_y) + abs(x - agent_x)
  pirate_dist = abs(x - prison_x) + abs(y - prison_y)
  if agent_dist > pirate_dist: return False
  return True

In [12]:
def generateHint7(map):

  choice = random.randint(0, 1)
  random_row_col = random.randint(0,len(map) - 1)
  # print(random_row_col)

  return [6, choice, random_row_col]

def verify_hint_7 (hint: list, treasurePos):
  # Hint 7: [6, 0: Row/ 1: Column, x]: A column and/or a row that contain the treasure (rare)
  x, y = treasurePos
  if hint[1] == 0:
    if hint[2] != x: return False
  else:
    if hint[2] != y: return False
  return True

In [13]:
# Hint 8: [7, 0: Row/ 1: Column, x]: A column and/or a row that do not contain the treasure.
def generateHint8(m: int):
    return [7, random.randint(0,1), random.randint(0,m-1)];

def verify_hint_8 (hint: list, treasurePos):
  # Hint 8: [7, 0: Row/ 1: Column, x]: A column and/or a row that do not contain the treasure.
  x, y = treasurePos
  if hint[1] == 0:
    if hint[2] == x: return False
  else:
    if hint[2] == y: return False
  return True

In [14]:
# Hint 9: [8, x, y]: 2 regions that the treasure is somewhere in their boundary
def generateHint9(map: list):
    boundary = set()
    m = len(map)

    xp = [-1,0,1,0]
    yp = [0,1,0,-1]

    def isInside(x: int, y: int, m: int) -> bool:
        return (0<=x<m and 0<=y<m)

    for x in range(m):
        for y in range(m):
            if (map[x][y]==0): continue
            for i in range(4):
                xt,yt = x+xp[i], y+yp[i]
                if (isInside(xt,yt,m) and map[xt][yt]!=0 and map[xt][yt]!=map[x][y]):
                    boundary.add((min(map[x][y], map[xt][yt]),max(map[x][y],map[xt][yt])))

    chosenBoundary = random.sample(list(boundary), 1)[0]
    print(boundary)
    print(chosenBoundary)
    return [8, chosenBoundary[0], chosenBoundary[1]]

def verifyHint9(hint: list, map: list, treasurePos) -> bool:
    x,y = treasurePos

    xp = [-1,0,1,0]
    yp = [0,1,0,-1]

    area1 = hint[1]
    area2 = hint[2]

    def isInside(x,y,m) -> bool:
        return (0<=x<m and 0<=y<m)

    for i in range(4):
        xt,yt = x+xp[i], y+yp[i]
        if (isInside(xt,yt,len(map)) and ((map[xt][yt]==area1 and map[x][y]==area2) or (map[xt][yt]==area2 and map[x][y]==area1))):
            return True

    return False

def isInside(x, y, m):
  return (0 <= x < m and 0 <= y < m)

In [15]:
# Hint 10: [9]: The treasure is somewhere in a boundary of 2 regions.
def generateHint10():
    return [9]

def verifyHint10(map: list, treasurePos) -> bool:
    xp = [-1, 0, 1, 0]
    yp = [0, 1, 0, -1]

    x, y = treasurePos

    for i in range(4):
        xt = x + xp[i]
        yt = y + yp[i]
        if (isInside(xt,yt,len(map)) and map[xt][yt]!=0 and map[xt][yt]!=map[x][y]):
            return True

    return False

In [16]:
# Hint 11: [10, 2/3]: The treasure is somewhere in an area bounded by 2-3 tiles from sea.
def generateHint11():
    return [10, random.randint(2,3)]

def verifyHint11(hint: list, map: list, treasurePos):
    x,y = treasurePos
    dist = hint[1]

    xp = [-1,0,1,0]
    yp = [0,1,0,-1]

    def isInside(x, y, m):
        return (0<=x<m and 0<=y<m)

    for i in range(4):
        xt = x+xp[i]*dist
        yt = y+yp[i]*dist
        if (isInside(xt,yt,len(map)) and map[xt][yt]==0):
            return True

    return False

In [17]:
# Hint 12: [11, 1: Half right / 2: Half left / 3: Half top / 4: Half bottom]: A half of the map without treasure (rare).
def generateHint12():
    return [11, random.randint(1,4)]

def verifyHint12(hint: list, mapSize, treasurePos):
    x,y = treasurePos

    pos = hint[1]

    if (pos==1):
        return y>=mapSize/2
    elif (pos==2):
        return y<mapSize/2
    elif (pos==3):
        return x<mapSize/2
    else:
        return x>=mapSize/2

treasurePos = [3,0]
hint = [11,3]
print(verifyHint12(hint, 4, treasurePos))

False


In [18]:
# Hint 13: [12, 1: Center / 2: Prison, 1: N / 2: S / 3: W / 4: E / 5: SE / 6: SW / 7: NE / 8: NW]:
# From the center of the map/from the prison that he's staying,
# he tells you a direction that has the treasure (W, E, N, S or SE, SW, NE, NW)
# (The shape of area when the hints are either W, E, N or S is triangle).

def generateHint13():
    return [12, random.randint(1, 2), random.randint(1, 8)]

def verifyHint13(hint: list, mapSize, prisonPos, treasurePos) -> bool:
    treasureX,treasureY = treasurePos
    pos, direction = hint[1], hint[2]

    # if from center
    if (pos==1):
        half = mapSize//2
        # if north
        if (direction==1):
            if (treasureX>=half):
                return False
            dist = half-treasureX
            return half-dist<=treasureY<=half+dist-1
        # if south
        elif (direction==2):
            if (treasureX<half):
                return False
            dist = treasureX-half
            return half-dist<=treasureY<=half+dist-1
        # if west
        elif (direction==3):
            if (treasureY>=half):
                return False
            dist = half-treasureY
            return half-dist<=treasureX<=half+dist-1
        # if east
        elif (direction==4):
            if (treasureY<half):
                return False
            dist = treasureY-half
            return half-dist<=treasureX<=half+dist-1
        # if south east
        elif (direction==5):
            return (half<=treasureX and half<=treasureY)
        # if south west
        elif (direction==6):
            return (half<=treasureX and treasureY<half)
        # if north east
        elif (direction==7):
            return (treasureX<half and half<=treasureY)
        # if north west
        else:
            return (treasureX<half and treasureY<half)
    else:
        prisonX, prisonY = prisonPos
        # if north
        if (direction==1):
            if (treasureX>=prisonX):
                return False
            dist = prisonX-treasureX
            return prisonY-dist<=treasureY<=prisonY+dist
        # if south
        elif (direction==2):
            if (treasureX<prisonX):
                return False
            dist = treasureX-prisonX
            return prisonY-dist<=treasureY<=prisonY+dist
        # if west
        elif (direction==3):
            if (treasureY>=prisonY):
                return False
            dist = prisonY-treasureY
            return prisonX-dist<=treasureX<=prisonX+dist
        # if east
        elif (direction==4):
            if (treasureY<prisonY):
                return False
            dist = treasureY-prisonY
            return prisonX-dist<=treasureX<=prisonX+dist
        # if south east
        elif (direction==5):
            return (prisonX<=treasureX and prisonY<=treasureY)
        # if south west
        elif (direction==6):
            return (prisonX<=treasureX and treasureY<prisonY)
        # if north east
        elif (direction==7):
            return (treasureX<prisonX and prisonY<=treasureY)
        # if north west
        else:
            return (treasureX<prisonX and treasureY<prisonY)

In [19]:
# Hint 14: [13, top_big, bottom_big, left_big, right_big, top_small, bottom_small, left_small, right_small]: 
# 2 squares that are different in size, the small one is placed inside the bigger one, the treasure is somewhere inside the gap between 2 squares. (rare)
def generateHint14(mapSize: int):
    topBig = random.randint(0,mapSize-3)
    bottomBig = random.randint(topBig+2,mapSize-1)
    leftBig = random.randint(0,mapSize-3)
    rightBig = random.randint(leftBig+2,mapSize-1)

    # print(f'{topBig}, {leftBig}, {bottomBig}, {rightBig}')

    bigSize = min(bottomBig-topBig, rightBig-leftBig)    
    # print(f'{bigSize}')

    bottomBig = topBig+bigSize
    rightBig = leftBig+bigSize

    topSmall = random.randint(topBig+1,bottomBig-1)
    bottomSmall = random.randint(topSmall,bottomBig-1)
    leftSmall = random.randint(leftBig+1,rightBig-1)
    rightSmall = random.randint(leftSmall,rightBig-1)
    # print(f'{topSmall}, {leftSmall}, {bottomSmall}, {rightSmall}')

    smallSize = min(bottomSmall-topSmall, rightSmall-leftSmall)    
    # print(f'{smallSize}')
    
    bottomSmall = topSmall+smallSize
    rightSmall = leftSmall+smallSize
    return [13, topBig, bottomBig, leftBig, rightBig, topSmall, bottomSmall, leftSmall, rightSmall]

def verifyHint14(hint:list, treasurePos) -> bool:
    x, y = treasurePos

    topBig, bottomBig, leftBig, rightBig = hint[1],hint[2], hint[3], hint[4]
    topSmall, bottomSmall, leftSmall, rightSmall = hint[5], hint[6], hint[7], hint[8]
    return ((topBig<=x<topSmall or bottomSmall<x<=bottomBig) and leftBig<=y<=rightBig) or (topBig<=x<=bottomBig and (leftBig<=y<=leftSmall))


In [20]:
def verifyHint15(map: list, treasurePos) -> bool:
    x, y = treasurePos
    return map[x][y]=='M'

# Game Rule 

In [109]:
m=MapGenerator(size=16,num_of_region=5,num_of_prison=3,num_of_mountain=3)
m.visualize()

 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 
 0  1  0  0 1M 1M  1  1  2  0  0  0  0 2M  0  0 
 0  0  1  1 1M 1M  1  1  2  2  2  2 2M 2M  2  0 
 0  0  1  1  1 1M  1  1  2  2  2 2M 2M  2  2  0 
 0  1  1  1  1  1  1  1  2  2  2  2 2T  2  3  0 
 0  0  1  1  1  1  1 1P  2  2  2  2  2  2  3  0 
 0  0  1  1  1  1  1  1  2  2  2  2  2  2  0  0 
 0  1  1  1  1  1  4  2  2  2  2  2  2  2  5  0 
 0  0  4  4  4  4  4  4  4  4  4  5  5  5  0  0 
 0  0  4  4  4  4  4  4  4  4  5  5  5  5  5  0 
 0  0  4  4  4  4  4  4  4  4  5  5  5  5  0  0 
 0  0  4  4  4  4  4  4 4P  4  5  5  5  5  0  0 
 0  0  4  4  4  4  4  4  4  4 5M 5M 5M  5  0  0 
 0  0  4  4  4  4  4  4  4  4  5  5 5M  5  0  0 
 0  0  0  0 4P  4  4  0  0  4  0  0 5M  5  0  0 
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 


In [ ]:
class Game():
    def __init__(self,filename):
        with open (filename) as f:
            self.map=[]
            self.w,self.h=f.readline().replace('\n','').split(' ')
            self.w,self.h=int(self.w),int(self.h)
            self.reveal_prison_turn=int(f.readline().replace('\n',''))
            self.release_turn=int(f.readline().replace('\n',''))
            self.num_of_region=int(f.readline().replace('\n',''))
            self.treasure_pos=f.readline().replace('\n','').split(' ')
            self.treasure_pos=[int(self.treasure_pos[0]),int(self.treasure_pos[1])]
            for i in range(self.h):
                data=f.readline().replace('\n','').replace(' ','').split(';')
                self.map.append(data)
    def visualize_map(self):
        for i in range(len(self.map)):
            for j in range(len(self.map[i])):
                print('{:>2}'.format(self.map[i][j]),end=' ')
            print()
        
                
t=Game('MAP02.txt')

# Visualization

# Output